# Merolagani

> use this for getting floorsheet of a particular day

In [7]:
from selenium import webdriver
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import sys

In [8]:
def search(driver, date):
    """
    Date in mm/dd/yyyy
    """
    driver.get("https://merolagani.com/Floorsheet.aspx")
    date_input = driver.find_element_by_xpath('/html/body/form/div[4]/div[4]/div/div/div[1]/div[4]/input')
    search_btn = driver.find_element_by_xpath('/html/body/form/div[4]/div[4]/div/div/div[2]/a[1]')
    date_input.send_keys(date)
    search_btn.click()
    if driver.find_elements_by_xpath("//*[contains(text(), 'Could not find floorsheet matching the search criteria')]"):
        print("No data found for the given search.")
        print("Aborting script ......")
        sys.exit()

In [3]:
def get_page_table(driver, table_class):
    soup = BeautifulSoup(driver.page_source,'html')
    table = soup.find("table", {"class":table_class})
    tab_data = [[cell.text.replace('\r', '').replace('\n', '') for cell in row.find_all(["th","td"])]
                        for row in table.find_all("tr")]
    df = pd.DataFrame(tab_data)
    return df

In [4]:
def scrape_data(driver, date):
    start_time = datetime.now()
    search(driver, date = date)
    df = pd.DataFrame()
    while True:
        page_table_df = get_page_table(driver, table_class="table table-bordered table-striped table-hover sortable")
        df = df.append(page_table_df, ignore_index = True)
        try:
            next_btn = driver.find_element_by_link_text('Next')
            driver.execute_script("arguments[0].click();", next_btn)
        except NoSuchElementException:
            break
    print(f"Time taken to scrape: {datetime.now() - start_time}")    
    return df

In [5]:
def clean_df(df):
    new_df = df.drop_duplicates(keep='first') # Dropping Duplicates
    new_header = new_df.iloc[0] # grabing the first row for the header
    new_df = new_df[1:] # taking the data lower than the header row
    new_df.columns = new_header # setting the header row as the df header
    new_df.drop(["#"], axis=1, inplace=True)
    new_df["Rate"] = new_df["Rate"].apply(lambda x:float(x.replace(",", ""))) # Convert Rate to Float
    new_df["Amount"] = new_df["Amount"].apply(lambda x:float(x.replace(",", ""))) # Convert Amount to Float
    return new_df

In [6]:
options = Options()
options.headless = True
driver = webdriver.Chrome(chrome_options=options) # Start Browser

date = datetime.today().strftime('%m/%d/%Y') # Get today's date
search(driver, date) # Search the webpage
df = scrape_data(driver, date) # Scraping
final_df = clean_df(df) # Cleaning

/home/suyog/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


No data found for the given search.
Aborting script ......


SystemExit: 

/home/suyog/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
final_df.head()

In [ ]:
file_name = date.replace("/", "_")
final_df.to_csv(f"data/{file_name}.csv", index=False) # Save file

# Nepalweb

> use this to get data form nepalweb

In [16]:
import requests
import json
import pandas as pd
from fake_useragent import UserAgent
import time

In [16]:
url = 'https://newweb.nepalstock.com.np/api/nots/nepse-data/floorsheet?page=0&size=500&sort=contractId,desc'
r = requests.get(url, allow_redirects=True,headers=headers)

In [17]:
data = json.loads(r.content)

In [20]:
floorsheet = data['floorsheets']['content']

In [21]:
df = pd.DataFrame(floorsheet)

In [22]:
start_time = time.time()
i = 0
df = pd.DataFrame()
ua = UserAgent()

while True:
    headers = {"User-Agent": ua.random}
    url = f'https://newweb.nepalstock.com.np/api/nots/nepse-data/floorsheet?page={i}&size=500&sort=contractId,desc'
    try:
        r = requests.get(url, allow_redirects=True,headers=headers)
        data = json.loads(r.content)
        floorsheet = data['floorsheets']['content']
        if len(floorsheet)==0:
            break
        df_i = pd.DataFrame(floorsheet)
        df = df.append(df_i)
        i+=1
        time.sleep(0.2)
    except:
        time.sleep(1)
        continue
        
print("--- %s seconds ---" % (time.time() - start_time))

--- 162.43655681610107 seconds ---


In [23]:
df.head()

,id,contractId,contractType,stockSymbol,buyerMemberId,sellerMemberId,contractQuantity,contractRate,contractAmount,businessDate,tradeBookId,stockId,buyerBrokerName,sellerBrokerName,tradeTime,securityName
0,None,2021053105001360,None,KLBSL,37,16,80,1655.0,132400.0,2021-05-31,30215699,694,Swarna Laxmi Securities Pvt. Ltd.,Primo Securities (PVT) Ltd.,2021-05-31T14:59:59.679583,Kisan Lagubitta Bittiya Sanstha Limited
1,None,2021053105001359,None,SHL,58,38,100,240.0,24000.0,2021-05-31,30214551,147,Naasa Securities Co. Ltd.,Dipshikha Dhitopatra Karobar Co. Pvt Ltd.,2021-05-31T14:59:41.165067,Soaltee Hotel Limited
2,None,2021053105001358,None,KLBSL,7,7,50,1656.0,82800.0,2021-05-31,30213796,694,J.F. Securites (PVT) Ltd.,J.F. Securites (PVT) Ltd.,2021-05-31T14:59:30.475507,Kisan Lagubitta Bittiya Sanstha Limited
3,None,2021053105001357,None,SHL,43,38,161,240.0,38640.0,2021-05-31,30212646,147,South Asian Bulls Pvt. Ltd.,Dipshikha Dhitopatra Karobar Co. Pvt Ltd.,2021-05-31T14:59:12.062916,Soaltee Hotel Limited
4,None,2021053105001356,None,SHL,43,38,189,240.0,45360.0,2021-05-31,30212637,147,South Asian Bulls Pvt. Ltd.,Dipshikha Dhitopatra Karobar Co. Pvt Ltd.,2021-05-31T14:59:12.062148,Soaltee Hotel Limited


In [32]:
df

,id,contractId,contractType,stockSymbol,buyerMemberId,sellerMemberId,contractQuantity,contractRate,contractAmount,businessDate,tradeBookId,stockId,buyerBrokerName,sellerBrokerName,tradeTime,securityName
0,None,2021053104057836,None,SHPC,58,49,200,462.0,92400.0,2021-05-31,30187984,591,Naasa Securities Co. Ltd.,Online Securities Pvt. Ltd.,2021-05-31T14:47:53.755997,Sanima Mai Hydropower Ltd.
1,None,2021053104057835,None,SHPC,55,49,200,462.0,92400.0,2021-05-31,30187977,591,Bhrikuti Stock Broking Co. Pvt. Ltd.,Online Securities Pvt. Ltd.,2021-05-31T14:47:53.75564,Sanima Mai Hydropower Ltd.
2,None,2021053104057834,None,SHPC,16,49,889,462.0,410718.0,2021-05-31,30187969,591,Primo Securities (PVT) Ltd.,Online Securities Pvt. Ltd.,2021-05-31T14:47:53.75529,Sanima Mai Hydropower Ltd.
3,None,2021053104057833,None,SHPC,43,49,2000,463.0,926000.0,2021-05-31,30187962,591,South Asian Bulls Pvt. Ltd.,Online Securities Pvt. Ltd.,2021-05-31T14:47:53.754728,Sanima Mai Hydropower Ltd.
4,None,2021053104057832,None,SJCL,44,38,10,466.0,4660.0,2021-05-31,30187923,2842,Dynamic Money Managers Securities Pvt. Ltd.,Dipshikha Dhitopatra Karobar Co. Pvt Ltd.,2021-05-31T14:47:53.509024,SANJEN JALAVIDHYUT COMPANY LIMITED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,None,2021053104057341,None,BPCL,54,25,100,519.0,51900.0,2021-05-31,30185772,153,Sewa Securities Pvt. Ltd.,Sweta Securities Private Limited,2021-05-31T14:46:35.640875,Butwal Power Company Limited
496,None,2021053104057340,None,BPCL,26,25,161,519.0,83559.0,2021-05-31,30185753,153,Asian Securities Private Ltd.,Sweta Securities Private Limited,2021-05-31T14:46:35.640238,Butwal Power Company Limited
497,None,2021053104057339,None,AIL,19,58,10,923.0,9230.0,2021-05-31,30185743,2893,Nepal Investment And Securities Trading Privat...,Naasa Securities Co. Ltd.,2021-05-31T14:46:35.459655,Ajod Insurance Limited
498,None,2021053104057338,None,AKJCL,56,44,32,267.0,8544.0,2021-05-31,30185726,2788,Sri Hari Securities Pvt. Ltd.,Dynamic Money Managers Securities Pvt. Ltd.,2021-05-31T14:46:34.909941,Ankhu Khola Jalvidhyut Company Ltd
